<a href="https://colab.research.google.com/github/Sudhansh-Mehta/Recommender_Systems/blob/main/Collaborative_filtering_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [6]:
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'])
rating_df=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'])

In [7]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [8]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [9]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [11]:
rating_df.shape

(100836, 3)

In [12]:
movies_df.shape

(9742, 2)

In [13]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
 3   title    100836 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [15]:
movie_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

In [16]:
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [17]:
rating_with_totalRatingCount = df.merge(movie_ratingCount, on='title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [18]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()
#remved movies which had been rated fewer than 50 times , similarly users who rated less than 50 movies will also be removed

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [19]:
rating_popular_movie.shape

(41362, 5)

In [20]:
#pivot table
#left will be the things that you want to recommend
#top will be the ones you want to recommend to (the ones you are generating recommendations for)
# movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating',fill_value=0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,3,0,5.0,0.0,0,0.0,0,0.0
12 Angry Men (1957),0,0.0,0.0,5,0,0,0.0,0,0,0.0,...,5.0,0,0,0,0.0,0.0,0,0.0,0,0.0
2001: A Space Odyssey (1968),0,0.0,0.0,0,0,0,4.0,0,0,0.0,...,0.0,0,5,0,0.0,5.0,0,3.0,0,4.5
28 Days Later (2002),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,3.5,0,5.0
300 (2007),0,0.0,0.0,0,0,0,0.0,0,0,3.0,...,0.0,0,0,0,3.0,0.0,0,5.0,0,4.0


In [21]:
movie_features_df.shape

(450, 606)

In [22]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)


In [23]:
print(movie_features_df_matrix)  


  (0, 11)	5.0
  (0, 18)	3.0
  (0, 66)	4.5
  (0, 90)	5.0
  (0, 102)	5.0
  (0, 109)	4.0
  (0, 130)	4.0
  (0, 151)	1.0
  (0, 156)	3.5
  (0, 174)	4.5
  (0, 179)	4.0
  (0, 195)	1.0
  (0, 197)	5.0
  (0, 216)	2.0
  (0, 233)	5.0
  (0, 246)	3.5
  (0, 257)	4.5
  (0, 271)	3.0
  (0, 272)	4.0
  (0, 277)	4.0
  (0, 283)	3.0
  (0, 295)	2.5
  (0, 304)	0.5
  (0, 313)	3.0
  (0, 322)	3.0
  :	:
  (449, 378)	2.0
  (449, 384)	3.0
  (449, 410)	0.5
  (449, 411)	4.0
  (449, 416)	4.0
  (449, 417)	4.0
  (449, 421)	3.0
  (449, 423)	2.0
  (449, 435)	2.5
  (449, 444)	3.0
  (449, 476)	4.5
  (449, 479)	5.0
  (449, 480)	4.5
  (449, 485)	1.5
  (449, 491)	5.0
  (449, 521)	4.0
  (449, 530)	4.0
  (449, 538)	1.5
  (449, 556)	4.0
  (449, 557)	3.0
  (449, 569)	5.0
  (449, 585)	2.5
  (449, 595)	4.5
  (449, 603)	3.0
  (449, 605)	4.0


In [24]:
from sklearn.neighbors import NearestNeighbors


model_nn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_nn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
query_index = 1

distances, indices = model_nn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
print(distances)
print(indices)

[[0.         0.59553693 0.61265857 0.61814687 0.62013789 0.62491882]]
[[  1 298  80 221 292 403]]


In [26]:
movie_features_df.index[1]

'12 Angry Men (1957)'

In [28]:
indices.shape

(1, 6)

In [31]:
relevant_indices=indices.flatten()
print(relevant_indices)

[  1 298  80 221 292 403]


In [32]:
for i in relevant_indices:
  print(movie_features_df.index[i])
  #print(movie_features_df.index[i])

12 Angry Men (1957)
One Flew Over the Cuckoo's Nest (1975)
Casablanca (1942)
It's a Wonderful Life (1946)
North by Northwest (1959)
To Kill a Mockingbird (1962)
